# Source laoding, basic setup

In [7]:
!conda info --envs

# conda environments:
#
base                  *  /mmfs1/apps/easybuild/software/Miniforge3/24.1.2-0
                         /mmfs1/project/sa2564/dv336/fine/llm



In [1]:
!pwd

/project/sa2564/dv336/fine


In [8]:
!nvidia-smi

Fri Sep 20 00:51:18 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  |   00000000:41:00.0 Off |                    0 |
| N/A   26C    P0             60W /  500W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

# Begin

In [1]:
!nvidia-smi

Fri Sep 20 01:55:16 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  |   00000000:01:00.0 Off |                    0 |
| N/A   29C    P0             60W /  500W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
import torch
import torch.nn as nn
from accelerate import Accelerator
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
   bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


base_model = AutoModelForCausalLM.from_pretrained(
    "Qwen/CodeQwen1.5-7B-Chat",
    quantization_config=bnb_config,
    use_cache=False,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

# Model load

In [2]:
model_id = "Qwen/CodeQwen1.5-7B-Chat"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [3]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [4]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

lora_config = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj", "k_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

base_model = prepare_model_for_kbit_training(base_model)
model = get_peft_model(base_model, lora_config)
print_trainable_parameters(model)

trainable params: 35651584 || all params: 4039544832 || trainable%: 0.8825643849173154


In [5]:
from datasets import load_dataset, Dataset
train_data = load_dataset("json", data_files="beta_train_all.json")
#test_data = load_dataset("json", data_files=$testdatasetpath$)

In [6]:
print(train_data)

DatasetDict({
    train: Dataset({
        features: ['description', 'metrics', 'code'],
        num_rows: 4000
    })
})


In [7]:
def tokenize(prompt, add_eos_token=True):
    # there's probably a way to do this with the tokenizer settings
    # but again, gotta move fast
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=3000,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < 3000
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    result["labels"] = result["input_ids"].copy()
    #result["labels"] = torch.empty_like(result["input_ids"]).copy_(result["input_ids"])

    return result




def generate_prompt(example) -> str:
    messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant. Please generate the Verilog header by taking the needed description:",
        },
        {"role": "user", "content": example['description']},
    ]

    tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(model.device)
    prompt = tokenizer.decode(tokenized_chat[0])

    #code_str = "".join(example['code']) if isinstance(example['code'], list) else str(example['code'])
    final = f"{prompt}\n\n{str(example['code'])}"
    tokenized_full_prompt = tokenize(final)
    return tokenized_full_prompt

In [8]:
train_val = train_data["train"].train_test_split(
    test_size=500, shuffle=True, seed=42
)
train_data = (
    train_val["train"].shuffle().map(generate_prompt)
)
val_data = (
    train_val["test"].shuffle().map(generate_prompt)
)

Map:   0%|          | 0/3500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [9]:
train_data = train_data.remove_columns(train_data['train'].column_names)
val_data = val_data.remove_columns(train_data['train'].column_names)

KeyError: "Column train not in the dataset. Current columns in the dataset: ['description', 'metrics', 'code', 'input_ids', 'attention_mask', 'labels']"

In [10]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="finQwen7B",
    num_train_epochs=3, # replace this, depending on your dataset
    per_device_train_batch_size=1,
    learning_rate=1e-5,
    optim="adamw_torch",
    gradient_accumulation_steps=16,
    logging_steps=10,
    eval_strategy="steps",
    save_strategy="steps",
    eval_steps= 300,
    save_steps=300,
    save_total_limit=4,
    load_best_model_at_end=True
)

In [11]:
import torch
# Step 1: Import AdamW from torch.optim
optimizer = torch.optim.AdamW(model.parameters(), lr=args.learning_rate)

# Step 2: Initialize the accelerator
accelerator = Accelerator()

# Step 3: Prepare the model, optimizer, and dataloaders for multi-GPU or distributed training
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_data, val_data
)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [12]:
import transformers
trainer = transformers.Trainer(
    model=model,
    tokenizer=tokenizer,
    #lora_config= lora_config,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=args,
    data_collator=transformers.DataCollatorForSeq2Seq(
        tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
    ),
)

trainer.train()

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/mmfs1/project/sa2564/dv336/fine/llm_finetuning_venv/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/mmfs1/project/sa2564/dv336/fine/llm_finetuning_venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.a

/mmfs1/project/sa2564/dv336/fine/llm_finetuning_venv/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/mmfs1/project/sa2564/dv336/fine/llm_finetuning_venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/mmfs1/project/sa2564/dv336/fine/llm_finetuning_venv/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWa

TrainOutput(global_step=654, training_loss=1.1540225753361297, metrics={'train_runtime': 21648.4314, 'train_samples_per_second': 0.485, 'train_steps_per_second': 0.03, 'total_flos': 2.904775081253929e+17, 'train_loss': 1.1540225753361297, 'epoch': 2.9897142857142858})

In [13]:
from transformers import Trainer

trainer.save_model("./ft_CodeQwen_7b")

# FT model load

In [14]:
eval_results=trainer.evaluate()

In [16]:
eval_results

{'eval_loss': 0.4042148292064667,
 'eval_runtime': 271.7362,
 'eval_samples_per_second': 1.84,
 'eval_steps_per_second': 0.232,
 'epoch': 2.9897142857142858}